# 📡 XAI Semantic Communication - Training sur Colab

Ce notebook entraîne le modèle JSCC sur COCO (118k images) avec A100.

**Prérequis :**
- Dataset COCO dans `/content/drive/MyDrive/CocoData/train2017/`
- Runtime GPU A100 activé

## 1. Configuration de l'environnement

In [ ]:
# Vérifier le GPU
!nvidia-smi

In [ ]:
# Monter Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Vérifier le dataset COCO
import os
coco_path = '/content/drive/MyDrive/CocoData/train2017'
if os.path.exists(coco_path):
    n_images = len([f for f in os.listdir(coco_path) if f.endswith('.jpg')])
    print(f"✅ Dataset COCO trouvé : {n_images} images")
else:
    print("❌ Dataset non trouvé ! Vérifiez le chemin.")

## 2. Cloner le code

In [ ]:
# Option A : Cloner depuis GitHub (modifier l'URL)
# !git clone https://github.com/votre_user/semantic_comm.git

# Option B : Copier depuis Drive (si code zippé sur Drive)
!cp -r /content/drive/MyDrive/semantic_comm /content/semantic_comm

%cd /content/semantic_comm

In [ ]:
# Installer les dépendances
!pip install -q torch torchvision tqdm pillow

In [ ]:
# Vérifier la structure du projet
!ls -la
!ls src/models/

## 3. Configuration des chemins (Drive)

In [ ]:
import os

# Créer les dossiers de sauvegarde sur Drive
DRIVE_SAVE_DIR = '/content/drive/MyDrive/semantic_comm_results'
os.makedirs(f'{DRIVE_SAVE_DIR}/checkpoints', exist_ok=True)
os.makedirs(f'{DRIVE_SAVE_DIR}/results', exist_ok=True)

# Liens symboliques pour que le code sauvegarde directement sur Drive
!rm -rf /content/semantic_comm/checkpoints
!rm -rf /content/semantic_comm/results
!ln -s {DRIVE_SAVE_DIR}/checkpoints /content/semantic_comm/checkpoints
!ln -s {DRIVE_SAVE_DIR}/results /content/semantic_comm/results

print("✅ Checkpoints et résultats seront sauvegardés sur Drive")

## 4. Modifier le chemin du dataset

In [ ]:
# Patch du chemin dataset dans train_noskip.py
# Le dataset est maintenant sur Drive

train_file = '/content/semantic_comm/src/train/train_noskip.py'

with open(train_file, 'r') as f:
    content = f.read()

# Remplacer le chemin du dataset
old_path = "root_dir=str(project_root.parent / 'CocoData')"
new_path = "root_dir='/content/drive/MyDrive/CocoData/train2017'"

content = content.replace(old_path, new_path)

with open(train_file, 'w') as f:
    f.write(content)

print("✅ Chemin dataset mis à jour")

## 5. Lancer l'entraînement

In [ ]:
# Entraînement avec batch_size=32 (A100 peut gérer plus)
!python -m src.train.train_noskip --batch_size 32 --epochs 100 --lr 2e-4

## 6. Reprendre l'entraînement (si interrompu)

In [ ]:
# Si la session a été interrompue, relancer avec --resume
!python -m src.train.train_noskip --batch_size 32 --epochs 100 --lr 2e-4 --resume

## 7. Visualiser les métriques

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Charger les métriques
df = pd.read_csv('/content/semantic_comm/results/noskip/metrics.csv')
print(df.tail(10))

# Graphiques
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

axes[0, 0].plot(df['epoch'], df['grid_avg_psnr'])
axes[0, 0].set_title('Grid PSNR')
axes[0, 0].set_xlabel('Epoch')

axes[0, 1].plot(df['epoch'], df['grid_avg_ssim'])
axes[0, 1].set_title('Grid SSIM')
axes[0, 1].set_xlabel('Epoch')

axes[1, 0].plot(df['epoch'], df['train_budget'])
axes[1, 0].set_title('Budget Loss')
axes[1, 0].set_xlabel('Epoch')

axes[1, 1].plot(df['epoch'], df['mono_score'])
axes[1, 1].set_title('Mono Score')
axes[1, 1].set_xlabel('Epoch')

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/semantic_comm_results/training_curves.png')
plt.show()

## 8. Télécharger le meilleur modèle

In [ ]:
# Le modèle est déjà sur Drive grâce aux liens symboliques
!ls -la /content/drive/MyDrive/semantic_comm_results/checkpoints/
print("\n✅ Modèles sauvegardés sur Drive !")